In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import glob
import os

plt.style.use('ggplot') # Make the graphs a bit prettier

In [2]:
data = {}

for path in glob.glob(os.path.expanduser("~/dev/ma/data/results/pre_dlm1/*.json")):
  for typename, items in json.load(open(path)).items():
    if typename in data:
      data[typename].update(items)
    else:
      data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

graph_names = {
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000-sorted-preprocessed-*.bin': 'LFR 100K', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_1000000-preprocessed-*.bin': 'LFR 1M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_10000000-preprocessed-*.bin': 'LFR 10M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000000-preprocessed-*.bin': 'LFR 100M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_200000000-sorted-preprocessed-*.bin': 'LFR 200M', 
    '/home/kit/iti/kp0036/graphs/hypercubegraph23-preprocessed-*.bin': 'hypercube', 
    '/home/kit/iti/kp0036/graphs/uk-2002.metis-preprocessed-*.bin': 'uk-2002', 
    '/home/kit/iti/kp0036/graphs/uk-2007-05.metis-preprocessed-*.bin': 'uk-2007-05', 
    '/home/kit/iti/kp0036/graphs/in-2004.metis-preprocessed-*.bin': 'in-2004', 
    '/home/kit/iti/kp0036/graphs/com-friendster-preprocessed-*.bin': 'com-friendster', 
    '/home/kit/iti/kp0036/graphs/com-lj.ungraph-preprocessed-*.bin': 'com-lj', 
    '/home/kit/iti/kp0036/graphs/com-orkut.ungraph-preprocessed-*.bin': 'com-orkut', 
    '/home/kit/iti/kp0036/graphs/com-youtube.ungraph-preprocessed-*.bin': 'com-youtube', 
    '/home/kit/iti/kp0036/graphs/com-amazon.ungraph-preprocessed-*.bin': 'com-amazon'
}


frames['program_run'].replace({ 'graph': graph_names }, inplace=True)

In [3]:
frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['graph', 'local_moving_stopping_criterion', 'local_moving_node_ratio'])['modularity', 'runtime'] \
    .mean()

modularity  \
graph          local_moving_stopping_criterion local_moving_node_ratio               
LFR 10M        cluster_count                   1                          0.587117   
                                               2                          0.595163   
                                               4                          0.588008   
                                               8                          0.598188   
                                               dynamic                    0.574811   
               moved_count                     1                          0.593185   
                                               2                          0.596367   
                                               4                          0.596907   
                                               8                          0.587475   
                                               dynamic                    0.584092   
com-friendster cluster_count                   1                          0.505257   
                                               2                          0.574823   
                                               4                          0.577839   
                                               8                          0.591440   
                                               dynamic                    0.607118   
               moved_count                     2                          0.595428   
                                               4                          0.611349   
                                               8                          0.613112   
                                               dynamic                    0.624124   
uk-2002        cluster_count                   1                          0.348436   
                                               2                          0.988985   
                                               4                          0.989275   
                                               8                          0.989550   
                                               dynamic                    0.989080   
               moved_count                     2                          0.989374   
                                               4                          0.989609   
                                               8                          0.989660   
                                               dynamic                    0.989579   

                                                                            runtime  
graph          local_moving_stopping_criterion local_moving_node_ratio               
LFR 10M        cluster_count                   1                         144.988748  
                                               2                         198.402892  
                                               4                         283.782000  
                                               8                         912.742191  
                                               dynamic                   204.410172  
               moved_count                     1                         201.965941  
                                               2                         174.213303  
                                               4                         190.032780  
                                               8                         202.820913  
                                               dynamic                   220.911456  
com-friendster cluster_count                   1                         845.836941  
                                               2                         948.078753  
                                               4                        1319.555163  
                                               8                        2920.162724  
                                               dynamic                  1413.715154  
               moved_count   

In [4]:
df = frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['graph', 'local_moving_stopping_criterion', 'local_moving_node_ratio'])['modularity', 'runtime'] \
    .mean() \
    .stack() \
    .unstack(['local_moving_stopping_criterion', 'local_moving_node_ratio'])
  
df.columns = df.columns.rename(['stopping criterion', 'ratio'])

df

stopping criterion        cluster_count                                        \
ratio                                 1           2            4            8   
graph                                                                           
LFR 10M        modularity      0.587117    0.595163     0.588008     0.598188   
               runtime       144.988748  198.402892   283.782000   912.742191   
com-friendster modularity      0.505257    0.574823     0.577839     0.591440   
               runtime       845.836941  948.078753  1319.555163  2920.162724   
uk-2002        modularity      0.348436    0.988985     0.989275     0.989550   
               runtime        85.180454  124.188546   158.990083   450.134352   

stopping criterion                     moved_count                            \
ratio                          dynamic           1            2            4   
graph                                                                          
LFR 10M        modularity     0.574811    0.593185     0.596367     0.596907   
               runtime      204.410172  201.965941   174.213303   190.032780   
com-friendster modularity     0.607118         NaN     0.595428     0.611349   
               runtime     1413.715154         NaN  1967.538978  2620.497621   
uk-2002        modularity     0.989080         NaN     0.989374     0.989609   
               runtime       70.951985         NaN   149.457490   132.547036   

stopping criterion                                   
ratio                                8      dynamic  
graph                                                
LFR 10M        modularity     0.587475     0.584092  
               runtime      202.820913   220.911456  
com-friendster modularity     0.613112     0.624124  
               runtime     2298.872828  2741.186756  
uk-2002        modularity     0.989660     0.989579  
               runtime      114.781697   123.721434

In [5]:
deltas = df.copy()
for c in df.columns:
    deltas[c] = sum([(df[c] - df[d].fillna(df[c])) for d in df.columns])
deltas

stopping criterion        cluster_count                            \
ratio                                 1            2            4   
graph                                                               
LFR 10M        modularity     -0.030143     0.050317    -0.021233   
               runtime     -1284.382915  -750.241475   103.549605   
com-friendster modularity     -0.753175    -0.127081    -0.099943   
               runtime     -9462.912448 -8542.736140 -5199.448450   
uk-2002        modularity     -5.127621     0.637314     0.639927   
               runtime      -643.328989  -292.256161    20.957672   

stopping criterion                                  moved_count              \
ratio                                8      dynamic           1           2   
graph                                                                         
LFR 10M        modularity     0.080567    -0.153203    0.030534    0.062354   
               runtime     6393.151511  -690.168675 -714.610985 -992.137365   
com-friendster modularity     0.022469     0.163574         NaN    0.058361   
               runtime     9206.019603 -4352.008528         NaN  632.405881   
uk-2002        modularity     0.642405     0.638172         NaN    0.640821   
               runtime     2641.256087  -771.385216         NaN  -64.835668   

stopping criterion                                                
ratio                                4            8      dynamic  
graph                                                             
LFR 10M        modularity     0.067757    -0.026563    -0.060389  
               runtime     -833.942599  -706.061262  -525.155839  
com-friendster modularity     0.201653     0.217517     0.316625  
               runtime     6509.033669  3614.410532  7595.235883  
uk-2002        modularity     0.642930     0.643395     0.642660  
               runtime     -217.029751  -376.917805  -296.460169

In [8]:
df.transpose()

graph                         LFR 10M             com-friendster               \
                           modularity     runtime     modularity      runtime   
stopping criterion ratio                                                        
cluster_count      1         0.587117  144.988748       0.505257   845.836941   
                   2         0.595163  198.402892       0.574823   948.078753   
                   4         0.588008  283.782000       0.577839  1319.555163   
                   8         0.598188  912.742191       0.591440  2920.162724   
                   dynamic   0.574811  204.410172       0.607118  1413.715154   
moved_count        1         0.593185  201.965941            NaN          NaN   
                   2         0.596367  174.213303       0.595428  1967.538978   
                   4         0.596907  190.032780       0.611349  2620.497621   
                   8         0.587475  202.820913       0.613112  2298.872828   
                   dynamic   0.584092  220.911456       0.624124  2741.186756   

graph                         uk-2002              
                           modularity     runtime  
stopping criterion ratio                           
cluster_count      1         0.348436   85.180454  
                   2         0.988985  124.188546  
                   4         0.989275  158.990083  
                   8         0.989550  450.134352  
                   dynamic   0.989080   70.951985  
moved_count        1              NaN         NaN  
                   2         0.989374  149.457490  
                   4         0.989609  132.547036  
                   8         0.989660  114.781697  
                   dynamic   0.989579  123.721434